<a href="https://colab.research.google.com/github/holoho/section-project/blob/main/section2project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



프로젝트 목표
데이터셋을 사용하여 머신러닝 모델 설정 후 성능 확인 및 인사이트 도출한다.

데이터 출처: US.GOV

데이터 셋: 부트캠프 내 지정된 주제가 아닌 다른 임의의 주제를 적용

문제 정의





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!pip install category_encoders
#!pip install shap
#!pip install pdpbox
#!pip install eli5
#!pip install xgboost
#!pip install -U imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pdpbox.pdp as pdp
#import shap, eli5

#from category_encoders import OrdinalEncoder, TargetEncoder
#from eli5.sklearn import PermutationImportance
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import f1_score
#from sklearn.model_selection import train_test_split, 
#from sklearn.pipeline import make_pipeline, Pipeline
#from sklearn.preprocessing import StandardScaler
#from xgboost import XGBClassifier

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/complaints.csv")

In [ ]:
df.isnull().sum()

In [ ]:
df1 = df.drop(columns = ['Date received','Sub-product','Sub-issue','Consumer complaint narrative','ZIP code','Tags','Date sent to company','Consumer disputed?','Complaint ID']) 

In [ ]:
df1 = df1.dropna()

In [ ]:
df1

In [ ]:
df1['Product'].unique() # 18

In [ ]:
df1['Issue'].unique() # 158

In [ ]:
df1['Company public response'].unique() # 11

In [ ]:
df1['Company'].unique() # 4274

In [ ]:
df1['State'].unique() # 63

In [ ]:
df1['Consumer consent provided?'].unique() # 4

In [ ]:
df1['Submitted via'].unique() # 2 ,Delete

In [ ]:
df1['Company response to consumer'].unique() # 4

In [ ]:
df2 = df1.drop(columns = 'Submitted via') 

In [ ]:
data = df2.drop_duplicates()

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
#결측치 확인
[(x, data[x].isnull().sum()) for x in data.columns if data[x].isnull().any()]

In [ ]:
#중복값 확인
data.duplicated().sum()

타겟 라벨 변환 ('Yes': 1 / 'No': 0)

In [ ]:
data['Timely response?'].replace(['Yes', 'No'], ['1', '0'], inplace = True)
data['Timely response?'] = data['Timely response?'].astype('int64')

In [ ]:
features = data.drop(columns=['Timely response?']).columns   

In [ ]:
cat_columns = data.select_dtypes(['object']).columns
cat_columns

In [ ]:
#convert all categorical variables to numeric
data2 =data.copy()
data2[cat_columns] = data2[cat_columns].apply(lambda x: pd.factorize(x)[0])

In [ ]:
data.head()

In [ ]:
data2

In [ ]:
data2.loc[data2['Consumer consent provided?']==0]

In [ ]:
data.loc[95]

EDA

In [ ]:
data["Issue"].value_counts().reset_index()[:5] # 가장 많은 불만 이슈

In [ ]:
data["Company"].value_counts().reset_index()[:10] # 고객들이 가장 많은 complaints 가진 회사

In [ ]:
data["State"].value_counts().reset_index()[:10] # 고객들이 가장 많은 complaints 가진 주

In [ ]:
data["Company public response"].value_counts().reset_index()[:10] # 고객들이 회사에 대한 가장 많은 후기 TOP 10

In [ ]:
data["Product"].value_counts().reset_index()[:5] # 고객들의 불만이 많았던 product TOP 10

In [ ]:
plt.subplots(1,1)
sns.countplot(data.State, order=data.State.value_counts().iloc[:14].index)
plt.xticks(rotation= 45)
plt.title("Complaints by State Top15",color = 'blue',fontsize=15)
plt.show()

plt.subplots(1,1)
sns.countplot(data.Company, order=data.Company.value_counts().iloc[0:14].index)
plt.title("Complaints by Company Top15",color = 'blue',fontsize=15)
plt.xticks(rotation= 90)
plt.show()


plt.subplots(1,1)
sns.countplot(data.Issue, order=data.Issue.value_counts().iloc[:5].index)
plt.title("Complaints by Issue",color = 'blue',fontsize=15)
plt.xticks(rotation= 75)
plt.show()


plt.subplots(1,1)
sns.countplot(data.Product, order=data.Product.value_counts().iloc[:5].index)
plt.title("Complaints by Product",color = 'blue',fontsize=15)
plt.xticks(rotation= 75)
plt.show()

In [ ]:
sns.displot(x = 'Consumer consent provided?', hue = 'Timely response?', kind = 'kde', data = data2)

In [ ]:
sns.displot(x = 'Company public response', hue = 'Timely response?', kind = 'kde', data = data2)

In [ ]:
sns.displot(x = 'Product', hue = 'Timely response?', kind = 'kde', data = data2)

In [ ]:
response_y= data.loc[data['Timely response?']==1]
len(response_y)

In [ ]:
plt.figure(figsize=(15,6))
sns.set_theme(style='darkgrid')
sns.countplot(data=response_y,x=response_y['Consumer consent provided?'], palette="Set3")

In [ ]:
plt.figure(figsize=(15,6))
sns.set_theme(style='darkgrid')
sns.countplot(data=response_y,x=response_y['Company response to consumer'], palette="Set3")

In [ ]:
response_x= data.loc[data['Timely response?']==0]
len(response_x)

In [ ]:
plt.figure(figsize=(15,6))
sns.set_theme(style='darkgrid')
sns.countplot(data=response_x,x=response_x['Consumer consent provided?'], palette="Set3")

In [ ]:
plt.figure(figsize=(15,6))
sns.set_theme(style='darkgrid')
sns.countplot(data=response_x,x=response_x['Company response to consumer'], palette="Set3")

시나리오

고객센터에 일하는 직원이다. 응답율이 저조한 회사들을 파악하고 어떤 불만사항들인지 확인해보자

In [ ]:
#print(data[target].loc[data[target] == 1].count(),
#      data[target].loc[data[target] == 0].count())

In [ ]:
#data[features].shape

기준모델 baseline model 선정, 모델링 성능 평가

In [ ]:
# 타겟 데이터 범주의 비율 확인
y_train = train[target]
y_train.value_counts(normalize = True) # 불균형 클래스

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.countplot(x = y_train)

In [ ]:
target = 'Timely response?'
data[target].value_counts(normalize=True).plot.pie(autopct="%.3f%%")
print("Baseline Accuracy:", data[target].value_counts(normalize=True).max())

In [ ]:
from sklearn.model_selection import train_test_split
target = 'Timely response?'
train, test = train_test_split(data2, test_size = 0.2, stratify = data[target])
train, val = train_test_split(train, test_size=0.2, stratify=train[target], random_state=42)
print(train.shape, val.shape , test.shape)

In [ ]:
print("Train target distribution:\n", train[target].value_counts(normalize=True), "\n")
print("Val target distribution:\n", val[target].value_counts(normalize=True), "\n")
print("Test target distribution:\n", test[target].value_counts(normalize=True))

In [ ]:
major = y_train.mode()[0]
y_pred = [major] * len(y_train)

In [ ]:
 #기준모델의 성능 평가 (정확도 accuracy)
from sklearn.metrics import accuracy_score
print("기준모델 학습 정확도: ", accuracy_score(y_train, y_pred)) # 최다 클래스의 빈도 = 정확도

# 기준모델 학습 정확도:  약 95.66%

In [ ]:
# 검증 세트에서의 정확도
train, val = train_test_split(train, test_size = 0.2, stratify = train[target])
print(train.shape, val.shape)

In [ ]:
y_val = val[target]
major1 = y_val.mode()[0]
y_pred_val = [major1] * len(y_val)
print("기준모델 검증 정확도: ", accuracy_score(y_val, y_pred_val))
# 기준모델 검증 정확도: 약 95.65%

In [ ]:
X_train, y_train = train.drop(columns=[target]), train[target]
X_val, y_val = val.drop(columns=[target]), val[target]
X_test, y_test = test.drop(columns=[target]), test[target]

In [ ]:
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=987654321,
    random_state=42,
    n_jobs=-1,
    max_depth=5,
    learning_rate=0.1,
)
print(model)

In [ ]:
watchlist = [(X_train_encoded, y_train), (X_val_encoded, y_val)]

model.fit(
    X_train_encoded,
    y_train,
    eval_set=watchlist,
    early_stopping_rounds=50,  # 50 rounds 동안 성능 개선이 없으면 학습을 중지합니다.
)

In [ ]:
import matplotlib.pyplot as plt

results = model.evals_result()
train_error = results["validation_0"]["auc"]
val_error = results["validation_1"]["auc"]

plt.plot(train_error, label="Train")
plt.plot(val_error, label="Validation")
plt.ylabel("ROC-AUC")
plt.xlabel("Model Complexity (n_estimators)")
plt.legend()

In [ ]:
from sklearn.metrics import classification_report

print("검증 정확도", model.score(X_val_encoded, y_val))

print(
    classification_report(
        y_val, model.predict(X_val_encoded)
    )
)

#1-class의 Recall이 굉장히 높다. 이 모델은 대부분의 입력을 1으로 예측하는 모델이라는 뜻

In [ ]:
model.predict(X_val_encoded).sum() #1이라고 예측하는 횟수

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score


def get_roc_auc(model, X_val_encoded, y_val):
    if isinstance(model, XGBClassifier):
        y_pred_proba = model.predict_proba(
            X_val_encoded
        )[:, 1]
    else:
        y_pred_proba = model.predict_proba(X_val_encoded)[:, 1]

    print("ROC-AUC:", roc_auc_score(y_val, y_pred_proba))

    baseline_fpr, baseline_tpr, _ = roc_curve(y_val, [0] * len(y_val))
    fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)

    plt.plot(
        baseline_fpr, baseline_tpr, linestyle="--", color="green", label="Baseline"
    )
    plt.plot(fpr, tpr, color="blue", label="Model")
    plt.scatter(fpr, tpr, color="red")
    plt.legend()
    plt.title("ROC curve")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.show()

In [ ]:
get_roc_auc(model, X_val_encoded, y_val)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns


def plot_confusion_matrix(model, X_val_encoded, y_val):
    if isinstance(model, XGBClassifier):
        y_pred = model.predict(X_val_encoded)
    else:
        y_pred = model.predict(X_val_encoded)
    cm = confusion_matrix(y_val, y_pred)
    sns.heatmap(cm, annot=True, fmt="g")
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.show()
     

In [ ]:
plot_confusion_matrix(model, X_val_encoded, y_val)

In [ ]:
# 함수화 된 정리
def summarize_results(model, X_val_encoded, y_val):
    # 1. Train / Val Loss Plot
    if isinstance(model, XGBClassifier):
        results = model.evals_result()
        tgts = results["validation_0"].keys()

        for tgt in tgts:
            train_error = results["validation_0"][tgt]
            val_error = results["validation_1"][tgt]

            plt.plot(train_error, label="Train")
            plt.plot(val_error, label="Validation")
            plt.ylabel(tgt)
            plt.xlabel("Model Complexity (n_estimators)")
            plt.legend()
            plt.show()

    # 2. Accuracy / Classification Report
    print("검증 정확도", model.score(X_val_encoded, y_val))

    if isinstance(model, XGBClassifier):
        y_pred = model.predict(X_val_encoded)
    else:
        y_pred = model.predict(X_val_encoded)

    print(classification_report(y_val, y_pred))

    # 3. ROC-AUC
    get_roc_auc(model, X_val_encoded, y_val)

    # 4. Confusion Matrix
    plot_confusion_matrix(model, X_val_encoded, y_val)
     

In [ ]:
summarize_results(model, X_val_encoded, y_val)

In [ ]:
# sample수를 적은 수에 맞춤
from imblearn.under_sampling import RandomUnderSampler

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=987654321,
    random_state=42,
    n_jobs=-1,
    max_depth=5,
    learning_rate=0.1,
)

X_train_sampled, y_train_sampled = RandomUnderSampler(random_state=42).fit_resample(
    X_train_encoded, y_train
)
     

In [ ]:
# Undersampling 시 각 class의 샘플들이 동일한 비율로 샘플링된 것을 볼 수 있습니다.
y_train_sampled.value_counts()

In [ ]:
watchlist = [(X_train_sampled, y_train_sampled), (X_val_encoded, y_val)]

model.fit(
    X_train_sampled,
    y_train_sampled,
    eval_set=watchlist,
    early_stopping_rounds=50,  # 50 rounds 동안 성능 개선이 없으면 학습을 중지합니다.
)

summarize_results(model, X_val_encoded, y_val)

In [ ]:
from imblearn.over_sampling import SMOTE


model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=987654321,
    random_state=42,
    n_jobs=-1,
    max_depth=5,
    learning_rate=0.1,
)

X_train_sampled, y_train_sampled = SMOTE(random_state=42).fit_resample(
    X_train_encoded, y_train
)

In [ ]:
# 0-class의 데이터가 1-class의 데이터 수만큼 늘어난 것을 볼 수 있습니다.
y_train_sampled.value_counts()

In [ ]:
watchlist = [(X_train_sampled, y_train_sampled), (X_val_encoded, y_val)]

model.fit(
    X_train_sampled,
    y_train_sampled,
    eval_set=watchlist,
    early_stopping_rounds=50,  # 50 rounds 동안 성능 개선이 없으면 학습을 중지합니다.
)
y_train_sampled.value_counts()
summarize_results(model, X_val_encoded, y_val)

In [ ]:
from imblearn.combine import SMOTEENN

X_train_sampled, y_train_sampled = SMOTEENN(random_state=42).fit_resample(
    X_train_encoded, y_train
)

model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    n_estimators=987654321,
    random_state=42,
    n_jobs=-1,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=(y_train_sampled == 1).sum() / (y_train_sampled == 0).sum(),
)
# 0-class 데이터가 1-class 데이터만큼 생성되었다가 노이즈가 정리되었으므로, 결과적으로 각 타겟 데이터가 동일한 비율은 아니지만 0-class 데이터가 일부 새로 생성되었습니다.
y_train_sampled.value_counts()

In [ ]:
watchlist = [(X_train_sampled, y_train_sampled), (X_val_encoded, y_val)]

model.fit(
    X_train_sampled,
    y_train_sampled,
    eval_set=watchlist,
    early_stopping_rounds=50,  # 50 rounds 동안 성능 개선이 없으면 학습을 중지합니다.
)
summarize_results(model, X_val_encoded, y_val)

In [ ]:
!pip install beautifultable

In [ ]:
from beautifultable import BeautifulTable
table = BeautifulTable()
table.column_headers = ['Sampling Name', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC Score','Accuracy']
table.append_row(['Baseline', '0.96', '1', '0.98', '0.8570','0.9567'])
table.append_row(['Randomundersampling', '0.99', '0.71', '0.83', '0.8361','0.7162'])
table.append_row(['SMOTE', '0.97', '0.91', '0.94', '0.8079','0.8898'])
table.append_row(['SMOTEEN', '0.98', '0.83', '0.90', '0.8084','0.8179'])

print(table)

In [ ]:
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

pipe = make_pipeline(
    # TargetEncoder: 범주형 변수 인코더로, 타겟값을 특성의 범주별로 평균내어 그 값으로 인코딩
    TargetEncoder(min_samples_leaf=1, smoothing=1), 
    SimpleImputer(strategy='median'), 
    RandomForestRegressor(max_depth = 10, n_jobs=-1, random_state=2)
)

k = 3

scores = cross_val_score(pipe, X_train_sampled, y_train_sampled, cv=k, 
                         scoring='neg_mean_absolute_error')

print(f'MAE for {k} folds:', -scores)

print(-scores.mean(),'\n',scores.std())

In [ ]:
#oversampling 사용하기로 결정 제일 결과값이 좋다
X_train_sampled, y_train_sampled = SMOTE(random_state=42).fit_resample(
    X_train_encoded, y_train)

In [ ]:
# 분류 모델
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# 앙상블 모델
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
# from sklearn import svm
# from sklearn.svm import SVC, LinearSVC

# 모델 검정
from sklearn import metrics
from sklearn.metrics import roc_curve, auc , f1_score
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report

from scipy.stats import randint, uniform

# 검정결과 시각화
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 최적화
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve, validation_curve # 학습곡선, 검증곡선
from sklearn.model_selection import cross_val_score # 교차타당도

In [ ]:
logistic_r = LogisticRegression(
    solver = 'liblinear', 
    penalty = 'l2', 
    C = 1.0, 
    random_state = 42
    )

decision_tree = DecisionTreeClassifier(
    criterion = 'entropy', 
    # maX_features = 1,
    min_samples_leaf = 1,
    random_state = 42
    ) 

randomforestclf = RandomForestClassifier(
    criterion = 'gini', 
    n_estimators = 100, 
    random_state = 42
    )

xgboost = XGBClassifier(
    learning_rate = 1,
    max_depth = 10, 
    n_estimators = 100, 
    random_state = 42
    )

adaboost = AdaBoostClassifier(
    base_estimator = decision_tree,
    n_estimators = 200, 
    learning_rate = 0.1, 
    random_state = 42
    )

In [ ]:
models = [logistic_r, decision_tree]
labels = ['Logistic Regression', 'DecisionTreeClassifier']

for model, label in zip(models, labels):
    scores = cross_val_score(
        estimator = model, 
        X = X_train_sampled, y = y_train_sampled, 
        cv = 10, 
        scoring = 'roc_auc')
    print("ROC AUC: %0.3f (+/- %0.3f) [%s]" %(scores.mean(), scores.std(), label))
    # ROC AUC: 0.707 (+/- 0.019) [Logistic Regression]
    # ROC AUC: 0.928 (+/- 0.019) [DecisionTreeClassifier]

In [ ]:
models = [logistic_r, decision_tree]
labels = ['Logistic Regression', 'DecisionTreeClassifier']
linestyles = ['-', '--']
colors = ['orange', 'blue']

for model, label, color, ls in zip(models, labels, colors, linestyles):
    model.fit(X_train_sampled, y_train_sampled)
    pred_proba = model.predict_proba(X_val)[:,1]

    fpr, tpr, thresholds = roc_curve(y_val, pred_proba)
    roc = pd.DataFrame({
        'FPR(Fall-out)': fpr,
        'TPR(Recall)': tpr,
        'Threshold': thresholds
    })
    plt.plot(fpr, tpr, linestyle = ls, label = '%s (auc = %0.3f)' %(label, auc(fpr, tpr)))

plt.legend(loc = 'lower right')
plt.grid(alpha = 0.4)
plt.title('ROC curve (Voting)')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)')
plt.plot([0, 1], [0, 1], linestyle = ':', color = 'grey', linewidth = 2)

plt.show()

In [ ]:
def get_clf_eval(y_true, y_pred, y_pred_proba):
    print('Confusion Matrix :\n', confusion_matrix(y_true, y_pred)) # "오차행렬"
    print('Accuracy : ', accuracy_score(y_true, y_pred))
    print('Precision : ', precision_score(y_true, y_pred))
    print('Recall : ', recall_score(y_true, y_pred))
    print('f1 score : ', f1_score(y_true, y_pred))
    print('roc_auc score : ', roc_auc_score(y_true, y_pred_proba))

def get_model_eval(model, X_train, X_val, y_train, y_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val) # 예측 확률값
    y_pred_proba = model.predict_proba(X_val)[:,1] # 1의 확률 array
    get_clf_eval(y_val, y_pred, y_pred_proba)

In [ ]:
get_model_eval(logistic_r, X_train_sampled, X_val, y_train_sampled, y_val)
print('--------------------------------------------------------------')
get_model_eval(decision_tree, X_train_sampled, X_val, y_train_sampled, y_val)

In [ ]:
models = [decision_tree, randomforestclf]
labels = ['DecisionTreeClassifier', 'RandomForestClassifier']

for model, label in zip(models, labels):
    scores = cross_val_score(
        estimator = model, 
        X = X_train_sampled, y = y_train_sampled, 
        cv = 10, 
        scoring = 'roc_auc')
    print("ROC AUC: %0.3f (+/- %0.3f) [%s]" %(scores.mean(), scores.std(), label))
    # ROC AUC: 0.928 (+/- 0.019) [DecisionTreeClassifier]
    # ROC AUC: 0.978 (+/- 0.013) [RandomForestClassifier]

In [ ]:
models = [decision_tree, randomforestclf]
labels = ['DecisionTreeClassifier', 'RandomForestClassifier']
linestyles = ['-', '--']
colors = ['orange', 'blue']

for model, label, color, ls in zip(models, labels, colors, linestyles):
    model.fit(X_train_sampled, y_train_sampled)
    pred_proba = model.predict_proba(X_val)[:,1]

    fpr, tpr, thresholds = roc_curve(y_val, pred_proba)
    roc = pd.DataFrame({
        'FPR(Fall-out)': fpr,
        'TPR(Recall)': tpr,
        'Threshold': thresholds
    })
    plt.plot(fpr, tpr, linestyle = ls, label = '%s (auc = %0.3f)' %(label, auc(fpr, tpr)))

plt.legend(loc = 'lower right')
plt.grid(alpha = 0.4)
plt.title('ROC curve (RF)')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)')
plt.plot([0, 1], [0, 1], linestyle = ':', color = 'grey', linewidth = 2)

plt.show()

In [ ]:
get_model_eval(randomforestclf, X_train_sampled, X_val, y_train_sampled, y_val)

In [ ]:
models = [xgboost, adaboost]
labels = ['XGBClassifier', 'AdaBoostClassifier']

for model, label in zip(models, labels):
    scores = cross_val_score(
        estimator = model, 
        X = X_train_sampled, y = y_train_sampled, 
        cv = 10, 
        scoring = 'roc_auc')
    print("ROC AUC: %0.3f (+/- %0.3f) [%s]" %(scores.mean(), scores.std(), label))

# ROC AUC: 0.985 (+/- 0.010) [XGBClassifier]
# ROC AUC: 0.949 (+/- 0.013) [AdaBoostClassifier]

In [ ]:
models = [xgboost, adaboost]
labels = ['XGBClassifier', 'AdaBoostClassifier']
linestyles = ['-', '--']
colors = ['orange', 'blue']

for model, label, color, ls in zip(models, labels, colors, linestyles):
    model.fit(X_train_sampled, y_train_sampled)
    pred_proba = model.predict_proba(X_val)[:,1]

    fpr, tpr, thresholds = roc_curve(y_val, pred_proba)
    roc = pd.DataFrame({
        'FPR(Fall-out)': fpr,
        'TPR(Recall)': tpr,
        'Threshold': thresholds
    })
    plt.plot(fpr, tpr, linestyle = ls, label = '%s (auc = %0.3f)' %(label, auc(fpr, tpr)))

plt.legend(loc = 'lower right')
plt.grid(alpha = 0.4)
plt.title('ROC curve (RF)')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)')
plt.plot([0, 1], [0, 1], linestyle = ':', color = 'grey', linewidth = 2)

plt.show()

In [ ]:
get_model_eval(xgboost, X_train_sampled, X_val, y_train_sampled, y_val)
print('--------------------------------------------------------------')
get_model_eval(adaboost, X_train_sampled, X_val, y_train_sampled, y_val)

In [ ]:
table = BeautifulTable()
table.column_headers = ['Model Name', 'Precision', 'Recall', 'F1 Score', 'ROC AUC Score','Accuracy']
table.append_row(['Logistic Regression', '0.9651', '0.5940', '0.7354', '0.5862', '0.5912'])
table.append_row(['Decision Tree', '0.9649', '0.9059', '0.9345', '0.5899','0.8786'])
table.append_row(['RandomForest', '0.9635', '0.9229', '0.9428', '0.7136','0.8929'])
table.append_row(['XGBoost', '0.9677', '0.9327', '0.9499', '0.7884','0.9059'])
table.append_row(['AdaBoost', '0.9632', '0.9085', '0.9351', '0.5863','0.8793'])
print(table)
# logist


모델 선택: fl-score+ROC score + Accuracy 모두 고려했ㅇㄹ 때 XGBoost가 가장 이상적인 모델

In [ ]:
%%time
#Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost
classifier = xgboost.XGBClassifier()

params = {'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

rs_model=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

#너무 길어져서 적당한 양의 하이퍼파라미터만 사용
#model fitting
rs_model.fit(X_train_sampled, y_train_sampled)

In [ ]:
print("Optimal Hyperparameter:", rs_model.best_params_)
print("ROC_AUC:", rs_model.best_score_)
classifier = rs_model.best_estimator_

In [ ]:
from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,X_val_encoded,y_val,cv=10)
score

In [ ]:
print(cross_val_score(classifier,X_test_encoded,y_test))
y_test_pred = classifier.predict(X_test_encoded)
print(classification_report(y_test, y_test_pred))
plot_confusion_matrix(classifier, X_test_encoded, y_test)

In [ ]:
import shap
row = X_test.iloc[[1]]

explainer = shap.TreeExplainer(classifier)
row_processed = encoder.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row, 
    link='logit' # SHAP value를 확률로 변환해 표시합니다.
    )

In [ ]:
shap_values = explainer.shap_values(X_test.iloc[:500])
shap.summary_plot(shap_values, X_test.iloc[:500])

In [ ]:
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    classifier,
    scoring = 'roc_auc', 
    n_iter = 5
    )

permuter.fit(X_test, y_test)
feature_names = X_test.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

In [ ]:
eli5.show_weights(
    permuter, 
    top = None, 
    feature_names = feature_names
    )


In [ ]:
y_train.value_counts(normalize = True)

In [ ]:
ratio = 0.04/0.96 # 클래스 비율 0 / 1 (약 10 / 90)
ratio

In [ ]:
eval_set1 = [(X_train, y_train), 
            (X_val, y_val)]
eval_set2 = [(X_train_sampled, y_train_sampled), 
            (X_val, y_val)]

In [ ]:
from xgboost import XGBClassifier
model1 = XGBClassifier(n_estimators=1000, verbosity=0, n_jobs=-1, scale_pos_weight=ratio)
model1.fit(X_train, y_train, eval_set = eval_set1, eval_metric = 'auc', early_stopping_rounds=10)

In [ ]:
model2 = XGBClassifier(n_estimators=1000, verbosity=0, n_jobs=-1, scale_pos_weight = 1) # 0.5/0.5
model2.fit(X_train_sampled, y_train_sampled, eval_set = eval_set2, eval_metric = 'auc', 
          early_stopping_rounds=10)

In [ ]:
# from sklearn.metrics import roc_auc_score 1
class_index = 1
y_pred_proba1 = model1.predict_proba(X_test)[:, class_index]
print(f'Test AUC for class "{model1.classes_[class_index]}":')
print(roc_auc_score(y_test, y_pred_proba1)) # 범위는 0~1, 수치는 높을수록 좋다

In [ ]:
# from sklearn.metrics import roc_auc_score 2
class_index = 1
y_pred_proba2 = model2.predict_proba(X_test)[:, class_index]
print(f'Test AUC for class (Oversampling) "{model2.classes_[class_index]}":')
print(roc_auc_score(y_test, y_pred_proba2)) # 범위는 0~1, 수치는 높을수록 좋다

In [ ]:
# Confusion Matrix 확인1
# from sklearn.metrics import classification_report
y_test_pred1 = model1.predict(X_test)
print(classification_report(y_test, y_test_pred1))

In [ ]:
# Confusion Matrix 확인2
# from sklearn.metrics import classification_report
y_test_pred2 = model2.predict(X_test)
print(classification_report(y_test, y_test_pred2))

In [ ]:
# shap 그래프로 예측 설명
import xgboost
import shap

explainer = shap.TreeExplainer(model1)
shap_values = explainer.shap_values(row)

shap.initjs()
shap.force_plot(
    base_value = explainer.expected_value, 
    shap_values = shap_values, 
    features = row, 
    link='logit' # SHAP value를 확률로 변환해 표시합니다.
)

In [ ]:
# 예측을 shap 그래프를 통해 설명하는 함수
feature_names = row.columns
feature_values = row.values[0]
shaps = pd.Series(shap_values[0], zip(feature_names, feature_values))

pros = shaps.sort_values(ascending=False)[:3].index
cons = shaps.sort_values(ascending=True)[:3].index

print('responsed 예측에 대한 Positive 요인 Top 3 입니다:')
for i, pro in enumerate(pros, start=1):
    feature_name, feature_value = pro
    print(f'{i}. {feature_name} : {feature_value}')

print('\n')
print('Negative 요인 Top 3 입니다:')
for i, con in enumerate(cons, start=1):
    feature_name, feature_value = con
    print(f'{i}. {feature_name} : {feature_value}')

In [ ]:
def explain(row_number):
    positive_class = 1
    positive_class_index = 1

    # row 값을 변환합니다
    row = X_test.iloc[[row_number]]
    # row_processed = processor.transform(row)

    # 예측하고 예측확률을 얻습니다 
    pred = model1.predict(row)[0] # row_processed
    pred_proba = model1.predict_proba(row)[0, positive_class_index] # row_processed
    pred_proba *= 100
    if pred != positive_class:
        pred_proba = 100 - pred_proba

    # 예측결과와 확률값을 얻습니다
    print(f'이 고객 응답율에 대한 예측결과는 {pred} 으로, 확률은 {pred_proba:.0f}% 입니다.')
    
    # SHAP를 추가합니다
    shap_values = explainer.shap_values(row) # row_processed

    # Fully Paid에 대한 top 3 pros, cons를 얻습니다
    feature_names = row.columns
    feature_values = row.values[0]
    shaps = pd.Series(shap_values[0], zip(feature_names, feature_values))
    pros = shaps.sort_values(ascending=False)[:3].index
    cons = shaps.sort_values(ascending=True)[:3].index

    # 예측에 가장 영향을 준 top3
    print('\n')
    print('Positive 영향을 가장 많이 주는 3가지 요인 입니다:')
    
    evidence = pros if pred == positive_class else cons
    for i, info in enumerate(evidence, start=1):
        feature_name, feature_value = info
        print(f'{i}. {feature_name} : {feature_value}')

    # 예측에 가장 반대적인 영향을 준 요인 top1
    print('\n')
    print('Negative 영향을 가장 많이 주는 3가지 요인 입니다:')
    
    evidence = cons if pred == positive_class else pros
    for i, info in enumerate(evidence, start=1):
        feature_name, feature_value = info
        print(f'{i}. {feature_name} : {feature_value}')

    # SHAP
    shap.initjs()
    return shap.force_plot(
        base_value=explainer.expected_value, 
        shap_values=shap_values, 
        features=row, 
        link='logit'
    )

In [ ]:
explain(37)